In [1]:
from ase.calculators.vasp import Vasp2
from ase.io import read,write
import numpy as np
import logging
import os
from ase.eos import EquationOfState

#!rm -r bench_Si_autotune_ENCUT


# Load Structure and Set Output Directory

In [2]:
out_dir = 'bench_Si_autotune_volume'
sys = read("POSCAR.mp-27_Si")

# Start Log and Define Calc

In [3]:
if not os.path.isdir(out_dir+"/log"):
    os.system("mkdir -p {}/log".format(out_dir))
logging.basicConfig(format='%(asctime)s : %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p',
                   filename=out_dir+"/log/valet.log", filemode="w",level=logging.INFO)
calc = Vasp2(xc='PBE',kpts=(4,4,4),directory=out_dir,atoms=sys,encut=520)

# Methods

In [4]:
sys = read("POSCAR.mp-27_Si")

start_scan = 0.85
end_scan = 1.15
num_scans = 10

start_cell = sys.get_cell()

def set_volume_autotune_params(start_scan=None,end_scan=None,num_scans=None,
                               poscar_init=None,exclude_type=None,only_type=None):
    return start_scan,end_scan,num_scans,poscar_init,exclude_type,only_type,vols,ens

## Get Energy and Volume Lists

In [7]:
ens = []
vols = []
def do_volume_scan(start_scan=0.85,end_scan=1.15,num_scans=10,poscar_init=None,exclude_type=None,only_type=None):
    logging.info("Commencing volume scan...")
    print("Determining optimal volume...")
    start_scan,end_scan,num_scans,poscar_init,exclude_type,only_type,vols,ens 
    
    # Load POSCAR
    try:
        sys = read(poscar_init)
    except:
        logging.error("Initial POSCAR file could not be found at {}".format(poscar_init))
        print("Error: See error log for details.")
        return None
    logging.info("Loaded initial POSCAR file.")
    
    # Do volume scan
    logging.info("Performing volume scan.")
    for x in np.linspace(start_scan,end_scan,num_scans):
        sys.set_cell(x*start_cell,scale_atoms=True)
        sys.set_calculator(calc)
        ens.append(sys.get_potential_energy())
        vols.append(sys.get_volume())
    logging.info("Volume scan complete.")
    
    # Fit EoS
    logging.info("Fitting equations of state.")
    eos_types = "sjeos taylor murnaghan birch birchmurnaghan pouriertarantola p3 antonschmidt".split()

    if exclude_type != None:
        _ = [eos_types.remove(i) for i in exclude]
    elif only_type != None:
        eos_types = only_type
        
    eos_fits = {}
    for typ in eos_types:
        eos = EquationOfState(vols,ens,eos=typ)
        v, e, B = eos.fit()
        eos_fits[typ] = {'volume':v,'energy':e,'buld_modulus':B}
        
    # Rescale initial cell to optimized volume
    logging.info("Rescaling with optimized volume.")
    vol_avg = np.average([eos_fits[key]['volume'] for key in eos_fits.keys()])
    sys.set_cell(start_cell,scale_atoms=True)

    scale = sys.get_volume()/vol_avg
    sys.set_cell(scale*start_cell,scale_atoms=True)
    
    # Perform sc-step
    logging.info("Performing self-consistent step.")
    sys.set_calculator(calc)
    en = sys.get_potential_energy()
    
    # Save structure
    logging.info("Saving final POSCAR_vol.")
    write(os.path.join(out_dir,"POSCAR_vol"),sys)
    
    print("Done.")
    logging.info("Volume scan complete.")
    return sys


In [8]:

sys = do_volume_scan(poscar_init="POSCAR.mp-27_Si")
sys.get_volume()    

Determining optimal volume...
Done.


55.83097337727178